In [1]:
import numpy as np 

In [195]:
# check if sequence of subsquare sides is suitable for filling the big square.
# We start from the top-left corner at (0, 0), and proceed by always finding
# the top-left-most corner (top first)
# (We use numpy for fancy indexing and not much else)
# Indexing order is (down, right)
# We use labels for each sub-square, starting at 1.
# 0 denotes unfilled tiles.

def first_free_tile_idx(board):
    # return (r,c) of first zero
    return tuple(np.argwhere(board==0)[0])

def square_fill(seq, side, shortcut=False, board=None):
    # Return (code, filled_board) or (code, partially_filled_board)
    # return codes: 0 = can't fill, 1 = partial, 2 = full fill
    if board is None:
        board = np.zeros((side, side), dtype=int)
    assert board.shape == (side, side) 
    # a few sanity checks first:
    if shortcut:
        if np.max(seq) > side:
            return (0, board)
        total = np.sum([s*s for s in seq])
        if total != np.sum(board==0):
            return (0, board)
    l0 = board.max()+1
    for l, s in enumerate(seq):
        r0, c0 = first_free_tile_idx(board)
        r1, c1 = r0 + s, c0 + s
        if r1 > side or c1 > side or np.any(board[r0:r1, c0:c1]):
            return (0, board)
        board[r0:r1, c0:c1] = l + l0
    if np.any(board==0): # left with incomplete board
        return (1, board)
    return (2, board)

In [196]:
# Example: fill a 5x5 square using 7 smaller squares
r, b = square_fill([3, 2, 1, 1, 2, 1, 2, 1], 5, shortcut=False)
print 'Fill: ', r
print b

Fill:  2
[[1 1 1 2 2]
 [1 1 1 2 2]
 [1 1 1 3 4]
 [5 5 6 7 7]
 [5 5 8 7 7]]


## First attempt: bruteforce search
### 5x5, nmax=8:
- with shortcut: 0.7s
- w/o shortcut: 2.1s

### 7x7, nmax=8:
even checking all combinations with up to 8 sub-sqaures takes 60s - not very scalable!

In [146]:
import itertools

In [181]:
%%time
side = 5  # side of large square
nmax = 8  # number of smaller squares

for n in [nmax]:#range(1, nmax+1):
    gen = xrange(side) #xrange(side, 0, -1)
    gens = [gen]*n 

    sequences = itertools.product(*gens)
    for seq in sequences:
        r, b = square_fill(seq, side, shortcut=True)
        if r==2:
            break
    if r==2:
        print 'Found filling with %d squares:' % n 
        print b
        break
    else:
        print 'Can not fill with %d squares' % n 
    

Found filling with 8 squares:
[[1 2 3 4 4]
 [5 5 6 4 4]
 [5 5 7 7 7]
 [8 8 7 7 7]
 [8 8 7 7 7]]
CPU times: user 4.35 s, sys: 8 ms, total: 4.36 s
Wall time: 4.34 s


## Second attempt: recursive search

In [197]:
# Re-use part-filled board
r, b = square_fill([3, 2], 5)
print r
print b
r2, b2 = square_fill([2, 2], 5, board=b)
print r2
print b2

1
[[1 1 1 2 2]
 [1 1 1 2 2]
 [1 1 1 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
1
[[1 1 1 2 2]
 [1 1 1 2 2]
 [1 1 1 3 3]
 [4 4 0 3 3]
 [4 4 0 0 0]]


In [238]:
def find_fill(side, seq=[], nmax=100):
    if len(seq) > nmax:
        return False
    r, b = square_fill(seq, side, shortcut=False)
    if r == 2:
        return seq
    if r ==0:
        return False
    for n in range(1, side):
        f = find_fill(side, seq=seq+[n], nmax=nmax)
        if f:
            return f
    return False

In [240]:
s = find_fill(3, seq=[], nmax=6)
print s
r, b = square_fill(s, 3)
print b

[1, 1, 1, 1, 2, 1]
[[1 2 3]
 [4 5 5]
 [6 5 5]]


### 5x5
7: nope

8: yep

In [259]:
%%time
side = 5  # side of large square
nmax = 8  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[1, 1, 1, 2, 2, 1, 3, 2]
[[1 2 3 4 4]
 [5 5 6 4 4]
 [5 5 7 7 7]
 [8 8 7 7 7]
 [8 8 7 7 7]]
CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 54.2 ms


### 7x7
8: nope

9: yep!

In [249]:
%%time
side = 7  # side of large square
nmax = 9  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[1, 1, 3, 2, 2, 2, 4, 1, 3]
[[1 2 3 3 3 4 4]
 [5 5 3 3 3 4 4]
 [5 5 3 3 3 6 6]
 [7 7 7 7 8 6 6]
 [7 7 7 7 9 9 9]
 [7 7 7 7 9 9 9]
 [7 7 7 7 9 9 9]]
CPU times: user 556 ms, sys: 4 ms, total: 560 ms
Wall time: 558 ms


### 9x9
5: nope

6: yep!

In [256]:
%%time
side = 9  # side of large square
nmax = 6  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[3, 3, 3, 3, 6, 3]
[[1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [1 1 1 2 2 2 3 3 3]
 [4 4 4 5 5 5 5 5 5]
 [4 4 4 5 5 5 5 5 5]
 [4 4 4 5 5 5 5 5 5]
 [6 6 6 5 5 5 5 5 5]
 [6 6 6 5 5 5 5 5 5]
 [6 6 6 5 5 5 5 5 5]]
CPU times: user 812 ms, sys: 0 ns, total: 812 ms
Wall time: 809 ms


### 11x11
6: nope

7: nope

8: nope (23s)

9: nope (60s)

10: nope (150s)

11: yep (23s)

In [265]:
%%time
side = 11  # side of large square
nmax = 11  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[1, 1, 3, 2, 4, 2, 2, 4, 1, 7, 4]
[[ 1  2  3  3  3  4  4  5  5  5  5]
 [ 6  6  3  3  3  4  4  5  5  5  5]
 [ 6  6  3  3  3  7  7  5  5  5  5]
 [ 8  8  8  8  9  7  7  5  5  5  5]
 [ 8  8  8  8 10 10 10 10 10 10 10]
 [ 8  8  8  8 10 10 10 10 10 10 10]
 [ 8  8  8  8 10 10 10 10 10 10 10]
 [11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 10 10 10 10 10 10 10]]
CPU times: user 23.2 s, sys: 16 ms, total: 23.2 s
Wall time: 23.2 s


---

### 13x13
10: nope (8min)

11: yep!! (7.5min) 

12: yep (88s)

In [268]:
%%time
side = 13  # side of large square
nmax = 10  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

False
CPU times: user 8min 11s, sys: 204 ms, total: 8min 11s
Wall time: 8min 12s


In [267]:
%%time
side = 13  # side of large square
nmax = 11  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[3, 2, 2, 6, 1, 3, 4, 2, 1, 7, 6]
[[ 1  1  1  2  2  3  3  4  4  4  4  4  4]
 [ 1  1  1  2  2  3  3  4  4  4  4  4  4]
 [ 1  1  1  5  6  6  6  4  4  4  4  4  4]
 [ 7  7  7  7  6  6  6  4  4  4  4  4  4]
 [ 7  7  7  7  6  6  6  4  4  4  4  4  4]
 [ 7  7  7  7  8  8  9  4  4  4  4  4  4]
 [ 7  7  7  7  8  8 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]
 [11 11 11 11 11 11 10 10 10 10 10 10 10]]
CPU times: user 7min 30s, sys: 84 ms, total: 7min 30s
Wall time: 7min 30s


In [266]:
%%time
side = 13  # side of large square
nmax = 12  # number of smaller squares

s = find_fill(side, seq=[], nmax=nmax)
print s
if s:
    r, b = square_fill(s, side)
    print b

[1, 1, 1, 4, 6, 3, 3, 3, 1, 1, 7, 6]
[[ 1  2  3  4  4  4  4  5  5  5  5  5  5]
 [ 6  6  6  4  4  4  4  5  5  5  5  5  5]
 [ 6  6  6  4  4  4  4  5  5  5  5  5  5]
 [ 6  6  6  4  4  4  4  5  5  5  5  5  5]
 [ 7  7  7  8  8  8  9  5  5  5  5  5  5]
 [ 7  7  7  8  8  8 10  5  5  5  5  5  5]
 [ 7  7  7  8  8  8 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]
 [12 12 12 12 12 12 11 11 11 11 11 11 11]]
CPU times: user 1min 22s, sys: 24 ms, total: 1min 22s
Wall time: 1min 22s
